In [1]:
# !pip install jiwer -qq

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from predict import predict
import onnxruntime
import onnx
from pathlib import Path
import jiwer
import pandas as pd
from helper import get_files
from pdb import set_trace
from words2num import normalize
pd.set_option('max_colwidth', 400)

In [4]:
model = onnx.load("/content/output.onnx")
model = onnxruntime.InferenceSession(model.SerializeToString())

In [5]:
data_path = "/content/test-bahasa/"

In [6]:
txts = get_files(data_path,[".txt"],recurse=True)
txts

(#780) [PosixPath('/content/test-bahasa/iium-audio-iium-95.txt'),PosixPath('/content/test-bahasa/wattpad-audio-wattpad-655.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-almukhalis.txt'),PosixPath('/content/test-bahasa/wattpad-audio-wattpad-61.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-streaming-sendiri melalui bunyi berirama.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-alot.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-news-audio-243.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ090-000005.txt'),PosixPath('/content/test-bahasa/iium-audio-iium-49.txt'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ263-000005.txt')...]

In [7]:
# find data with number in it
nombor = ["satu","dua",'tiga','puluh','ratus','ribu'] # can add more text if want to filter more
filtered_wavs = []
for i in txts:
    text = i.read_text().split()
    got = any([i in nombor for i in text])
    if got:
        filtered_wavs.append(i.with_suffix(".wav"))

# Predict single or multiple file

In [8]:
preds,files,entropy,timesteps = predict(model,filtered_wavs)

100%|██████████| 37/37 [00:13<00:00,  2.66it/s]


In [12]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    data.append([label,normalize(i),i])

In [13]:
df = pd.DataFrame(data)
df.columns = ["label","norm_pred","pred"]
df.head(20)

,label,norm_pred,pred
0,studi ini juga mendapati sebanyak empat puluh tiga percent anak yang datang,studi ini juga mendapati sebanyak 43 persen anak yang datang,studi ini juga mendapati sebanyak empat puluh tiga persen anak yang datang
1,pengalaman aku aku satu geng nak makan tengahari beli makanan aku ambik lauk telur jela nak jimat kawan aku ini ambik lauk ayam,penggalaman aku aku 1 geng nak makan tengah hari boleh makanan aku ambil laut telur jelar na mjimat kawan aku ini ambil laut ayam,penggalaman aku aku satu geng nak makan tengah hari boleh makanan aku ambil laut telur jelar na mjimat kawan aku ini ambil laut ayam
2,mengikuti dua jam pelajaran bersama pak dadi rasanya libra ingin cepat cepat,mengikuti 2 jam pelajaran bersama 4 dadi rasanya libirat ingin cepat cepat,mengikuti dua jam pelajaran bersama empat dadi rasanya libirat ingin cepat cepat
3,setahun yang lalu ketika baru satu bulan semenjak libra sma libra sudah merasa,setahun yang lalu ketika baru 1 bulan semenjak liberat sma ibras sudah merasa,setahun yang lalu ketika baru satu bulan semenjak liberat sma ibras sudah merasa
4,baru dua langkah dari gedung sekolah tiba tiba yasmin yang berada ditengah,baru 2 langkah dari gedung sekolah tiba tiba ya simin yang berada di tengah,baru dua langkah dari gedung sekolah tiba tiba ya simin yang berada di tengah
5,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi,45 ahli belia yang dilantik itu menjadi penyambung legasi,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi
6,may berharap perjanjian itu akan selesai sebelum dua puluh dua mei,mai berharap penjanjian itu akan selesai sebelum 22 mai,mai berharap penjanjian itu akan selesai sebelum dua puluh dua mai
7,satu elemen utama teori ini adalah peranan elit politik,1 elemen utama teori ini adalah peranan elit politik,satu elemen utama teori ini adalah peranan elit politik
8,ipa tiga arahnya tepat mengenai punggung salah satu siswi dikelas tersebut,ipa 3 arahnya tepat mengenai punggung salah 1 sisui di kelas tersebut,ipa tiga arahnya tepat mengenai punggung salah satu sisui di kelas tersebut
9,gila cape banget kecepatan lari kamu kalau lagi panik nambah dua kali,kilah capit bangyit kecapatan lari kamu kalau lagi panik enambah 2 kali,kilah capit bangyit kecapatan lari kamu kalau lagi panik enambah dua kali


In [12]:
# word to num normalization still wip, let use know if found a bug so we can fix it ASAP
# TODO: fix ip address digit spelling space